In [ ]:
!mkdir OpenImages

In [ ]:
cd OpenImages

/content/OpenImages


In [ ]:
!wget https://s3.amazonaws.com/fast-ai-imageclas/food-101.tgz

--2020-12-24 12:32:15--  https://s3.amazonaws.com/fast-ai-imageclas/food-101.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.80.110
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.80.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5686607260 (5.3G) [application/x-tar]
Saving to: ‘food-101.tgz’

food-101.tgz        100%[===================>]   5.30G  48.1MB/s    in 1m 58s  

2020-12-24 12:34:14 (45.8 MB/s) - ‘food-101.tgz’ saved [5686607260/5686607260]



In [ ]:
import tensorflow as tf 
import numpy as np
import tarfile
my_tar = tarfile.open('/content/OpenImages/food-101.tgz')
my_tar.extractall('/content/drive/MyDrive/Food_Images') # specify which folder to extract to
my_tar.close()

In [ ]:
import tensorflow as tf 
import numpy as np
train_ds = tf.keras.preprocessing.image_dataset_from_directory('/content/drive/MyDrive/Food_Images/food-101/images',
                                                    image_size = (224,224),
                                                    validation_split = 0.7, subset = 'training', seed = 1234, batch_size = 64)
test_ds = tf.keras.preprocessing.image_dataset_from_directory('/content/drive/MyDrive/Food_Images/food-101/images',
                                                    image_size = (224,224),
                                                    validation_split = 0.7, subset = 'validation', seed = 1234)

Found 101000 files belonging to 101 classes.
Using 30300 files for training.
Found 101000 files belonging to 101 classes.
Using 70700 files for validation.


In [ ]:
IMG_size = 224
IMG_shape = (IMG_size, IMG_size, 3)
base_model  = tf.keras.applications.InceptionV3(input_shape=IMG_shape,
                                                include_top = False, 
                                                weights = 'imagenet')
base_model.trainable = False

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
global_avg_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction = tf.keras.layers.Dense(101)

In [ ]:
model = tf.keras.models.Sequential([base_model, global_avg_layer, prediction])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 101)               206949    
Total params: 22,009,733
Trainable params: 206,949
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
base_learning_rate =0.01
model.compile(optimizer=tf.keras.optimizers.Adam(lr = base_learning_rate),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics = ['accuracy'])

In [ ]:
# Creating checkpoints
import os
# Directory where the checkpoints will be saved
checkpoint_dir = '/content/drive/MyDrive/Food_Images/checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
# Reloading the model
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
model.fit(train_ds, epochs = 20, callbacks=[checkpoint_callback])

Epoch 1/20
947/947 [==============================] - 7213s 8s/step - loss: 21.5266 - accuracy: 0.0686
Epoch 2/20
947/947 [==============================] - 148s 156ms/step - loss: 19.8751 - accuracy: 0.0785
Epoch 3/20
947/947 [==============================] - 149s 157ms/step - loss: 18.7308 - accuracy: 0.0831
Epoch 4/20
947/947 [==============================] - 149s 157ms/step - loss: 18.5603 - accuracy: 0.0906
Epoch 5/20
947/947 [==============================] - 150s 157ms/step - loss: 17.8213 - accuracy: 0.0946
Epoch 6/20
947/947 [==============================] - 149s 157ms/step - loss: 17.4719 - accuracy: 0.0977
Epoch 7/20
947/947 [==============================] - 151s 159ms/step - loss: 17.4553 - accuracy: 0.1036
Epoch 8/20
947/947 [==============================] - 151s 158ms/step - loss: 17.2796 - accuracy: 0.1069
Epoch 9/20
947/947 [==============================] - 152s 160ms/step - loss: 16.8996 - accuracy: 0.1125
Epoch 10/20
947/947 [==============================] - 15

In [ ]:
history = model.fit(train_ds, epochs = 100, callbacks=[checkpoint_callback])

Epoch 1/100
947/947 [==============================] - 17807s 19s/step - loss: 15.2041 - accuracy: 0.1644
Epoch 2/100
947/947 [==============================] - 148s 156ms/step - loss: 15.3591 - accuracy: 0.1703
Epoch 3/100
947/947 [==============================] - 149s 156ms/step - loss: 15.0388 - accuracy: 0.1672
Epoch 4/100
947/947 [==============================] - 149s 157ms/step - loss: 15.1602 - accuracy: 0.1701
Epoch 5/100
947/947 [==============================] - 149s 157ms/step - loss: 15.0074 - accuracy: 0.1737
Epoch 6/100
947/947 [==============================] - 149s 156ms/step - loss: 15.0224 - accuracy: 0.1764
Epoch 7/100
947/947 [==============================] - 149s 157ms/step - loss: 15.6799 - accuracy: 0.1717
Epoch 8/100
947/947 [==============================] - 149s 156ms/step - loss: 15.1195 - accuracy: 0.1779
Epoch 9/100
947/947 [==============================] - 149s 157ms/step - loss: 15.1951 - accuracy: 0.1752
Epoch 10/100
947/947 [========================